# For experiment

In [1]:
!pip install fredapi

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import preprocessor as pre
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np
import matplotlib.pyplot as plt
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

In [54]:
test = pd.read_csv('Data_flat_files/Dataset_Future_Predicting_Price_Evolutions_202403.csv')

price_evo_df = pre.clean_pred_price_evo_csv('Data_flat_files/Dataset_Future_Predicting_Price_Evolutions_202403.csv',2016,2023)

filter = (price_evo_df['Year'] == 2016) & (price_evo_df['Month'] == 1) & (price_evo_df['Group Description'] == "acid")

print(price_evo_df[filter].groupby(['Group Description','Key RM code']).size())
print(price_evo_df[filter].groupby(['Group Description','Key RM code']).mean())

Group Description  Key RM code
acid               RM01/0001      8
                   RM01/0004      5
                   RM01/0006      2
dtype: int64
                               PRICE (EUR/kg)    Year  Month
Group Description Key RM code                               
acid              RM01/0001          0.528015  2016.0    1.0
                  RM01/0004          0.404800  2016.0    1.0
                  RM01/0006          0.846000  2016.0    1.0


In [61]:
gas_df = pre.get_Fred_data('PNGASEUUSDM',2015,2023)
wheat_df = pre.get_Fred_data('PWHEAMTUSDM',2015,2023)
ammonia_df = pre.get_Fred_data('WPU0652013A',2015,2023)

price_evo_df = pre.clean_pred_price_evo_csv('Data_flat_files/Dataset_Future_Predicting_Price_Evolutions_202403.csv',2016,2023)
dummy_df = pre.get_dummies_and_average_price(price_evo_df,'acid',\
                                          'RM01/0001','RM01/0004','RM01/0006','RM01/0007')

acid_df = pre.generate_features(1,12,dummy_df,\
                                PNGASEUUSDM=gas_df,\
                                PWHEAMTUSDM=wheat_df,\
                                WPU0652013A=ammonia_df
                               )
acid_df.to_excel('Barry02Apr.xlsx')
print(dummy_df[filter].groupby(['Year','Month','RM01/0004','RM01/0006','RM01/0007']).size())

Year  Month  RM01/0004  RM01/0006  RM01/0007
2016  1      0          0          0            3
                        1          0            1
             1          0          0            2
      2      0          0          0            2
             1          0          0            1
      3      0          0          0            1
      4      1          0          0            2
      5      0          0          0            3
dtype: int64


In [5]:
test = pd.read_excel('Data_flat_files/acid_df.xlsx')
test['Year'] = [date.split('-')[0] for date in test['acid_ym'].values]
test['Month'] = [date.split('-')[1] for date in test['acid_ym'].values]
test = test.drop(['Unnamed: 0','acid_ym'], axis=1)
test.rename(columns={'cat_04':'RM01/0004',\
                     'cat_06':'RM01/0006',\
                     'cat_07':'RM01/0007',\
                     'acid_price':'Average_price'},\
           inplace=True)
# print(test.info())




In [6]:
xiao = (test[test['Year'] > "2015"])
barry = (acid_df[acid_df['Time'] < '2023-11-01'])

filter_1 = ((barry['RM01/0004'] == 0) &\
          (barry['RM01/0006'] == 0) &\
          (barry['RM01/0007'] == 0))

filter_2 = ((xiao['RM01/0004'] == 0) &\
          (xiao['RM01/0006'] == 0) &\
          (xiao['RM01/0007'] == 0))

# print(xiao.groupby(["Year"]).size())
# print(barry.groupby(['Year']).size())
# print(barry[filter_1].groupby(['Year'])['Average_price'].mean())
# print(xiao[filter_2].groupby(['Year'])['Average_price'].mean())

print(barry['Average_price'].mean())
print(xiao['Average_price'].mean())

0.7375364504620943
0.7289299381066436


In [27]:
purchase = pd.read_csv('Data_flat_files/Dataset_Future_Predicting_Price_Evolutions_202403.csv')
print(purchase.groupby(['Group Description']).size())

Group Description
Alkalis                 7502
Bleaching agent         5570
Solvent                  329
acid                    3812
anionic surfactant      1652
builder                 1876
fatty acid               888
non-ionic surfactant    4096
solvent                  422
dtype: int64


In [ ]:
# Import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Preprocessing 

# Import purchase data and price index data; rename columns

purchase = pd.read_csv("/Users/hexiaomeng/Documents/KU Leuven - IM/Thesis/Materials/Dataset_Predicting_Price_Evolutions.csv",
                            index_col=0)
purchase_data = purchase.loc[:,["POSTING DATE","Group Description","PRICE (EUR/kg)"]]
purchase_data.columns = ['date', 'group_description', 'acid_price']

# Filter purchase_data to contain only acid data
acid_data = purchase_data [purchase_data['group_description'] == 'acid'] # 3181 rows * 4 columns #

# Transform the timestamp into date, year, month, year_month
acid_data['date'] = pd.to_datetime(acid_data['date'])
acid_data['year'] = acid_data['date'].dt.year
acid_data['month'] = acid_data['date'].dt.month
acid_data['acid_ym'] = acid_data['year'].astype(str) + "-" + acid_data['month'].astype(str).str.zfill(2)
acid_data.sort_values(by = 'date', ascending = True, inplace = True)
acid_data.index = range(0, 3181)

# Subset acid_data to contain data from 2015-12-01 to 2023-12-01
acid_2015 = acid_data [acid_data['date'] > '2015-12-01'] # 2589 rows x 7 columns #
acid_2015.index = range(0, 2589)

# Aggregate daily purchase data into monthly data in acid_2015

acid_month = acid_2015.groupby(['acid_ym'])['acid_price'].mean()
acid_month = acid_month.to_frame()
acid_month = acid_month.reset_index() 
print (acid_month) # 95 rows x 2 columns #

# Merge all dataframes

acid_df = acid_month.merge(ele_data, on = 'acid_ym')
acid_df = acid_df.merge(gas_data, on = 'acid_ym')
acid_df = acid_df.merge(wheat_data, on = 'acid_ym')
acid_df = acid_df.merge(ammonia_data, on = 'acid_ym') 
print (acid_df) # 95 rows x  6 columns #
